In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [ ]:
# [pin]
file_path = "../data/data-hard.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
4233,⚠️🇷🇺#RASP акции «Распадской» снизились после ...,129-2,clsorg
3743,​​Падение цен на сталь на фоне замедления Кита...,90-2;116-2;152-2,clsorg
6468,📣 Полюс завершил свой выкуп Сегодня объявили ...,127-4,clsorg
912,"""🇷🇺#YNDX """"Яндекс"""" заявил, что утечка части ...",236-4,clsorg
5824,💥🇷🇺#ETLN = +7%,218-3,clsorg
4575,❗️🇷🇺#торги #россия СПБ Биржа с 13 марта измен...,225-3;230-3;251-3,clsorg
999,"""🏁 Итоги дня, 23.01 🍗 Группа """"Черкизово"""" п...",58-4;236-4,clsorg
4463,❗️ Мосбиржа (MOEX): справедливая цена 95 рубле...,103-1,clsorg
2812,Ожидаемые события на 31 января 🇷🇺 В России Р...,258-3,clsorg
2092,Акционеры Фосагро (PHOR) утвердили 465 руб./ак...,187-4,clsorg


In [ ]:
# m_name = "t5-small"
m_name = "cointegrated/rut5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

In [ ]:
from src.t5.dataset import NERDataModel

BATCH_SIZE = 128
EPOCHS = 10
num_workers = 12
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
data_module = NERDataModel(
    train_df, test_df, tokenizer, batch_size=BATCH_SIZE, num_workers=num_workers
)
data_module.setup()

In [ ]:
import torch
from transformers import T5ForConditionalGeneration

In [ ]:
m_name = "t5-small"
trained_model = T5ForConditionalGeneration.from_pretrained(m_name, return_dict=True)
state_dict = torch.load("./checkpoints/ner-v8.ckpt")["state_dict"]
state_dict = {k.partition("model.")[2]: v for k, v in state_dict.items()}
trained_model.load_state_dict(state_dict)
trained_model.save_pretrained("./pretrained")

In [ ]:
m_name = "./pretrained"
trained_model = T5ForConditionalGeneration.from_pretrained(
    m_name, return_dict=True, torch_dtype=torch.float16
)
trained_model.save_pretrained("./pretrained-fp16")

In [ ]:
m_name = "./pretrained-fp16"
trained_model = T5ForConditionalGeneration.from_pretrained(
    m_name, return_dict=True, torch_dtype=torch.float16
)
trained_model.cuda();

In [ ]:
import onnxruntime as rt
from optimum.onnxruntime import ORTModelForSeq2SeqLM

print(rt.get_available_providers())

ort_model = ORTModelForSeq2SeqLM.from_pretrained(
    "./pretrained-fp16",
    provider="CUDAExecutionProvider",
    export=True,
)

In [ ]:
import torch

from src.t5.utils import evaluate_metric, generate_answer_batched

In [ ]:
# [pin]
with torch.inference_mode(), torch.cuda.amp.autocast():
    predictions = generate_answer_batched(
        trained_model=ort_model,
        tokenizer=tokenizer,
        data=test_df[:],
        batch_size=128,
        num_beams=1,
        max_source_length=396,
        max_target_length=40,
    )

  0%|          | 0/15 [00:00<?, ?it/s]/home/worker/workspace/hakaton-gagarin-sentiment_interface/.conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|██████████| 15/15 [00:11<00:00,  1.28it/s]


In [ ]:
ldf = test_df.copy()[:]
ldf["predictions"] = predictions

ldf.loc[
    ldf[ldf.predictions.str.findall(r"[^\d\-;]").str.len() > 0].index, "predictions"
] = "0-3"

ldf[["tcomp", "tsent"]] = (
    ldf["target_text"].str.split(";", expand=True)[0].str.split("-", expand=True)
)

In [ ]:
from src.t5.utils import postprocess_predictions

orgsent = list(
    map(lambda x: x[0] if len(x) else [0, 1], postprocess_predictions(predictions))
)
org = list(map(lambda x: x[0], orgsent))
sent = list(map(lambda x: x[1], orgsent))

In [ ]:
# [pin]

evaluate_metric(
    company_predictions=ldf["pcomp"].tolist(),
    company_labels=ldf["tcomp"].tolist(),
    sentiment_predictions=ldf["psent"].tolist(),
    sentiment_labels=ldf["tsent"].tolist(),
)

{'total': 59.70054136860907,
 'f1': 0.6425361473499221,
 'accuracy': 0.5514746800222593}